In [ ]:
prime = [2,3,5,7]
iterator = iter(prime)
print(type(iterator))
next(iterator)

In [ ]:
def double_and_print(x):
    print(x, "->", 2 * x)
    return 2 * x


s = range(3, 7)
doubled = map(double_and_print, s)

In [ ]:
print(next(doubled))


In [ ]:
def letters_generator():
    current = "a"
    while current <= "g":
        yield current
        current = chr(ord(current) + 1)

In [ ]:
for letter in letters_generator():
    print(letter)

In [4]:
# 可迭代示例 1：一次性的街边小摊（卖完就没有了，迭代后耗尽）
class OneTimeStall:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self  # 摊主是自己的迭代器——卖完就没了

    def __next__(self):
        if self.current > self.n:
            raise StopIteration
        value = self.current * self.current
        self.current += 1
        return value

stall = OneTimeStall(5)
print("街边小摊开始卖东西：")
for v in stall:
    print(f"摊主拿出：{v}", end="  ")
print("\n顾客回头再来，摊位已经空了：")
for v in stall:
    print(f"摊主拿出：{v}", end="  ")
print("\n顾客回头再来，摊位已经空了：")

街边小摊开始卖东西：
摊主拿出：0  摊主拿出：1  摊主拿出：4  摊主拿出：9  摊主拿出：16  摊主拿出：25  
顾客回头再来，摊位已经空了：

顾客回头再来，摊位已经空了：


In [9]:
# 现实编程示例：使用 yield 生成批次（batch）用于流式处理或模型训练，节省内存
def batch_generator(iterable, batch_size):
    it = iter(iterable)
    batch = []
    for item in it:
        batch.append(item)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

# 模拟一个很大的数据流（range 是惰性生成，不会一次性占用内存）
large_stream = range(1_000_000)

# 逐批处理（这里只演示前 3 批以免输出过多）
for idx, b in enumerate(batch_generator(large_stream, 100_000)):
    print(f"处理第 {idx+1} 批，大小 = {len(b)}, 前 3 个样例 = {b[:3]}")
    if idx >= 2:
        break


处理第 1 批，大小 = 100000, 前 3 个样例 = [0, 1, 2]
处理第 2 批，大小 = 100000, 前 3 个样例 = [100000, 100001, 100002]
处理第 3 批，大小 = 100000, 前 3 个样例 = [200000, 200001, 200002]


In [ ]:
def oh_shit(n):
    for i in range(n):
        s = "oh, " + ("s" * (i + 1)) + "hit!"
        print(s)
        yield s


O = oh_shit(4)
next(O)
# >>> oh, shit!
print(type(O))
# >>> <class 'generator'>
print(type(oh_shit))
# >>> <class 'function'>

list(oh_shit(4))
# >>> oh, shit!
# >>> oh, sshit!
# >>> oh, ssshit!
# >>> oh, sssshit!
# >>> ['oh, shit!', 'oh, sshit!', 'oh, ssshit!', 'oh, sssshit!']

oh, shit!
<class 'generator'>
<class 'function'>
oh, shit!
oh, sshit!
oh, ssshit!
oh, sssshit!


['oh, shit!', 'oh, sshit!', 'oh, ssshit!', 'oh, sssshit!']

In [12]:
# 更易理解的例子：早安、数字、晚安
def greet_generator(x):
    yield "早安！"  # 第一次产出
    for i in range(x):
        yield f"数字 {i}"  # 依次产出数字
    yield "晚安！"  # 最后产出


# 使用
for item in greet_generator(3):
    print(item)
    print("-" * 20)  # 分隔线，便于观察顺序

早安！
--------------------
数字 0
--------------------
数字 1
--------------------
数字 2
--------------------
晚安！
--------------------


In [15]:
def main_generator():
    yield "oh, shit!"
    yield from greet_generator(4)
    yield "fine."

for item in main_generator():
    print(item)
    print("-" * 20)  # 分隔线，便于观察顺序

oh, shit!
--------------------
早安！
--------------------
数字 0
--------------------
数字 1
--------------------
数字 2
--------------------
数字 3
--------------------
晚安！
--------------------
fine.
--------------------


In [17]:
nested_list = [[1, 2, 3], ["A", "B", "C"], ["Jesus", "Christ"]]


def flatten_old(a_list):
    print("开始扁平化...")
    for sublist in a_list:
        for item in sublist:
            yield item
    print("扁平化结束。")


# 使用
for value in flatten_old(nested_list):
    print(value, end=" ")
# 输出: 1 2 3 A B C 4 5
print("-" * 20)


def flatten_new(a_list):
    print("开始扁平化...")
    for sublist in a_list:
        yield from sublist  # 关键在这里！
    print("扁平化结束。")


# 使用
for value in flatten_new(nested_list):
    print(value, end=" ")
# 输出: 1 2 3 A B C 4 5

开始扁平化...
1 2 3 A B C Jesus Christ 扁平化结束。
--------------------
开始扁平化...
1 2 3 A B C Jesus Christ 扁平化结束。


In [20]:
def greet():
    yield "Hello!"
    yield "欢迎使用本系统。"


def report_weather():
    # 假设这里有复杂的逻辑
    yield "天气预报：今日晴朗。"
    yield "气温：25°C。"


def say_goodbye():
    yield "感谢使用。"
    yield "再见！"


# --- 把它们串联起来 ---


def main_task():
    yield from greet()  # 委托给 greet 生成器
    yield from report_weather()  # greet 完成后，委托给 report_weather
    yield from say_goodbye()  # report_weather 完成后，委托给 say_goodbye


# 使用
for line in main_task():
    print(line)
    print("-" * 20)

Hello!
--------------------
欢迎使用本系统。
--------------------
天气预报：今日晴朗。
--------------------
气温：25°C。
--------------------
感谢使用。
--------------------
再见！
--------------------


In [3]:
def averager():
    """一个计算平均值的协程"""
    total = 0.0
    count = 0
    average = None
    print("计算器已启动...")
    while True:
        # yield 会暂停，并等待外部通过 send() 发送值过来
        # term 就是接收到的值
        term = yield average
        if term is None:  # 特殊情况，处理退出
            break
        total += term
        count += 1
        average = total / count
    return f"最终结果: 共 {count} 个数, 平均值为 {average:.2f}"


# -----------------------------------------------------------------


def main_coroutine():
    """主协程，委派任务给 averager"""
    print("主协程开始...")
    # yield from 会建立一个双向通道
    # 外部对 main_coroutine 的 send() 会直接传给 averager
    result = yield from averager()
    print(result)  # 打印 averager 的返回值
    print("主协程结束。")


# -----------------------------------------------------------------

# 1. 创建主协程对象
coro = main_coroutine()
# 2. 启动协程，执行到第一个 yield 处
next(coro)  # 或者 coro.send(None)
# 3. 通过 send() 发送数据，这些数据会穿过 yield from 直达 averager
print(f"发送 10, 当前平均值: {coro.send(10)}")
print(f"发送 20, 当前平均值: {coro.send(20)}")
print(f"发送 30, 当前平均值: {coro.send(30)}")

# 4. 发送 None 来关闭 averager 并获取返回值
try:
    coro.send(None)
except StopIteration:
    pass

主协程开始...
计算器已启动...
发送 10, 当前平均值: 10.0
发送 20, 当前平均值: 15.0
发送 30, 当前平均值: 20.0
最终结果: 共 3 个数, 平均值为 20.00
主协程结束。


In [7]:
class ElemIter:
    def __init__(self, lst):
        self.lst = lst
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.lst):
            result = self.lst[self.index]
            self.index += 1
            return result
        else:
            raise StopIteration

OP = ElemIter([1, 2, 3, 4, 5])
print(OP.__next__())
print("-"*20)
print(next(OP))

1
--------------------
2
